# Developing an Energy Model with Calliope tool

### In this notebook, I’m building a Calliope model using dummy data as a starting point. Later, I will incorporate weather data to compare the results against a mathematical model.

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import calliope
calliope.set_log_verbosity('INFO', include_solver_output=False)

In [15]:
df= pd.read_csv("calliope_model/timeseries/demand.csv", index_col=0)

In [16]:
df.head()

,demand
timestamp,
2020-01-01 00:00:00,0.3
2020-01-01 01:00:00,0.3
2020-01-01 02:00:00,0.3
2020-01-01 03:00:00,0.3
2020-01-01 04:00:00,0.3


In [17]:
df.dtypes

demand    float64
dtype: object

In [18]:
df['demand']=df['demand'] * -1

In [19]:
df.to_csv("calliope_model/timeseries/demand_copy.csv")

In [2]:
model=calliope.Model('calliope_model/model.yaml')


[2025-05-23 18:49:48] INFO     Model: initialising
[2025-05-23 18:49:49] INFO     `battery` at `mijn_stad` has no constraint to explicitly connect `energy_cap` to `storage_cap`, consider defining a `energy_cap_per_storage_cap_min/max/equals` constraint
[2025-05-23 18:49:49] WARNING  C:\Users\csten\anaconda3\envs\calliope\lib\site-packages\calliope\exceptions.py:60: ModelWarning:

Possible issues found during model processing:
 * Cost classes `{'monetary'}` are defined in the objective options but not defined elsewhere in the model. They will be ignored in the objective function.


[2025-05-23 18:49:49] INFO     Model: preprocessing stage 1 (model_run)
[2025-05-23 18:49:49] INFO     Model: preprocessing stage 2 (model_data)
[2025-05-23 18:49:49] INFO     Model: preprocessing complete


In [3]:
# Individual data variables can be accessed easily, `to_pandas()` reformats the data to look nicer
model.inputs.resource.to_pandas()

timesteps,2020-01-01 00:00:00,2020-01-01 01:00:00,2020-01-01 02:00:00,2020-01-01 03:00:00,2020-01-01 04:00:00,2020-01-01 05:00:00,2020-01-01 06:00:00,2020-01-01 07:00:00,2020-01-01 08:00:00,2020-01-01 09:00:00,...,2020-01-02 14:00:00,2020-01-02 15:00:00,2020-01-02 16:00:00,2020-01-02 17:00:00,2020-01-02 18:00:00,2020-01-02 19:00:00,2020-01-02 20:00:00,2020-01-02 21:00:00,2020-01-02 22:00:00,2020-01-02 23:00:00
loc_techs_finite_resource,,,,,,,,,,,,,,,,,,,,,
mijn_stad::demand_electricity,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.5,-0.7,...,-1.7,-1.9,-2.1,-2.3,-2.5,-2.7,-0.3,-0.3,-0.3,-0.3
mijn_stad::pv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# To reformat the array, deconcatenating loc_techs / loc_tech_carriers, you can use model.get_formatted_array()
# You can then apply loc/tech/carrier only operations, like summing information over locations: 
model.get_formatted_array('resource').sum('locs').to_pandas()

timesteps,2020-01-01 00:00:00,2020-01-01 01:00:00,2020-01-01 02:00:00,2020-01-01 03:00:00,2020-01-01 04:00:00,2020-01-01 05:00:00,2020-01-01 06:00:00,2020-01-01 07:00:00,2020-01-01 08:00:00,2020-01-01 09:00:00,...,2020-01-02 14:00:00,2020-01-02 15:00:00,2020-01-02 16:00:00,2020-01-02 17:00:00,2020-01-02 18:00:00,2020-01-02 19:00:00,2020-01-02 20:00:00,2020-01-02 21:00:00,2020-01-02 22:00:00,2020-01-02 23:00:00
techs,,,,,,,,,,,,,,,,,,,,,
demand_electricity,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,-0.5,-0.7,...,-1.7,-1.9,-2.1,-2.3,-2.5,-2.7,-0.3,-0.3,-0.3,-0.3
pv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
model.inputs

<xarray.Dataset>
Dimensions:                    (loc_techs_finite_resource: 2, timesteps: 48,
                                loc_techs_store: 1, loc_techs: 3, techs: 3,
                                loc_carriers: 2, loc_techs_non_conversion: 3,
                                locs: 1, carriers: 2)
Coordinates:
  * carriers                   (carriers) <U11 'electricity' 'power'
  * loc_carriers               (loc_carriers) object 'mijn_stad::power' 'mijn...
  * loc_techs                  (loc_techs) object 'mijn_stad::demand_electric...
  * loc_techs_finite_resource  (loc_techs_finite_resource) object 'mijn_stad:...
  * loc_techs_non_conversion   (loc_techs_non_conversion) object 'mijn_stad::...
  * loc_techs_store            (loc_techs_store) object 'mijn_stad::battery'
  * locs                       (locs) <U9 'mijn_stad'
  * techs                      (techs) object 'demand_electricity' ... 'pv'
  * timesteps                  (timesteps) datetime64[ns] 2020-01-01 ... 2020...
Data variables: (12/17)
    resource                   (loc_techs_finite_resource, timesteps) float64 ...
    storage_loss               (loc_techs_store) float64 0.01
    energy_cap_max             (loc_techs) float64 nan 10.0 20.0
    energy_prod                (loc_techs) float64 nan 1.0 1.0
    force_resource             (loc_techs_finite_resource) float64 1.0 nan
    resource_unit              (loc_techs_finite_resource) object 'energy' 'e...
    ...                         ...
    lookup_loc_carriers        (loc_carriers) object 'mijn_stad::battery::pow...
    lookup_loc_techs           (loc_techs_non_conversion) object 'mijn_stad::...
    lookup_loc_techs_area      (locs) object 'mijn_stad::pv'
    timestep_resolution        (timesteps) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    timestep_weights           (timesteps) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    max_demand_timesteps       (carriers) datetime64[ns] 2020-01-01T19:00:00 ...
Attributes:
    calliope_version:    0.6.10
    applied_overrides:   
    scenario:            None
    defaults:            carrier_ratios:\ncharge_rate:\nenergy_cap_per_storag...
    allow_operate_mode:  1

In [6]:
model.run()

[2025-05-23 18:49:54] INFO     Backend: starting model run
[2025-05-23 18:49:54] INFO     Loading sets
[2025-05-23 18:49:54] INFO     Loading parameters


AttributeError: 'ConcreteModel' object has no attribute 'costs'

## To be continued